In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
import json

# Load the trained model
model = load_model(r'D:\MODELs\inception80%.h5')

# Load average embeddings from the JSON file
with open("average_embeddings.json", "r") as f:
    average_embeddings = json.load(f)

# Initialize MTCNN for face detection
detector = MTCNN()

# Define a function to compute the embedding for an image array
def get_embedding_from_array(model, img_array):
    img_array = cv2.resize(img_array, (224, 224))  # Resize to model input size
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch
    embedding = model.predict(img_array)
    return embedding.flatten()


In [2]:
# Define a function to check identification status based on cosine similarity
def identify_or_predict(face_embedding, face_img, threshold=0.8):
    max_similarity = 0
    best_match = "Not Identified"
    
    # Case 1: Compute cosine similarity with average embeddings
    for class_name, avg_embedding in average_embeddings.items():
        avg_embedding = np.array(avg_embedding)
        similarity = cosine_similarity([face_embedding], [avg_embedding])[0][0]
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = class_name if similarity >= threshold else "Not Identified"
    
    # Case 2: If similarity is below threshold, perform prediction on the cropped face image
    if best_match == "Not Identified":
        # Resize the cropped face image to match the model's expected input shape
        target_size = (224, 224)  # Adjust based on your model's input size
        face_img_resized = cv2.resize(face_img, target_size)
        
        # Normalize and add batch dimension if necessary
        face_img_resized = face_img_resized / 255.0  # Normalize if the model expects it
        face_img_resized = np.expand_dims(face_img_resized, axis=0)  # Shape should be (1, height, width, channels)
        
        # Use the model to predict
        try:
            predicted_label = model.predict(face_img_resized)
            best_match = predicted_label[0] if predicted_label else "Not Identified"
        except Exception as e:
            print(f"Prediction error: {e}")
            best_match = "Not Identified"
    
    return best_match, max_similarity

# Start the webcam feed and perform face detection
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect faces in the frame
    faces = detector.detect_faces(frame)
    
    for face in faces:
        # Get the bounding box of the face
        x, y, width, height = face['box']
        x, y = abs(x), abs(y)
        
        # Extract face region
        face_img = frame[y:y + height, x:x + width]
        
        # Compute embedding for the detected face
        face_embedding = get_embedding_from_array(model, face_img)
        
        # Determine identity or "Not Identified" status
        best_match, similarity = identify_or_predict(face_embedding, face_img)
        
        # Display the result
        label = f"{best_match} ({similarity:.2f})"
        color = (0, 255, 0) if best_match != "Not Identified" else (0, 0, 255)
        
        cv2.rectangle(frame, (x, y), (x + width, y + height), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # Show the frame
    cv2.imshow("Live Face Recognition", frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Prediction error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Prediction error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Prediction error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediction error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Prediction error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.al

KeyboardInterrupt: 